# Installing Dependencies

In [ ]:
%pip install numpy matplotlib opencv-python scikit-learn tensorflow keras

# Importing Libraries

In [22]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os

# Paths

In [23]:
data_dir = "Published_database_FV-USM_Dec2013"
weights_path = "Weights"

# Importing Data

In [24]:
image_paths = []
labels = []

# Iterate through session folders
for session_folder in os.listdir(data_dir):
    session_path = os.path.join(data_dir, session_folder)
    
    # Iterate through raw_data or extractedvein folders
    for data_type_folder in os.listdir(session_path):
        if data_type_folder == 'extractedvein':
            data_type_path = os.path.join(session_path, data_type_folder)

            # Iterate through individual folders
            for individual_folder in os.listdir(data_type_path):
                individual_path = os.path.join(data_type_path, individual_folder)

                # Iterate through images
                for image_file in os.listdir(individual_path):
                    image_path = os.path.join(individual_path, image_file)
                    image_paths.append(image_path)
                    labels.append(int(individual_folder[4:7]))

In [25]:
image_paths = np.array(image_paths)
labels = np.array(labels)

# Test Train Validate Split

In [26]:
from sklearn.model_selection import train_test_split

train_ratio = 0.7
val_ratio = 0.15
test_ratio = 0.15

train_img_paths, remaining_paths, train_labels, remaining_labels = train_test_split(image_paths, labels, 
                                                                                    train_size=train_ratio, random_state=42,
                                                                                   stratify=labels)
val_to_test_ratio = val_ratio / (val_ratio + test_ratio)
# Split remaining set into validation and test sets
val_img_paths, test_img_paths, val_labels, test_labels = train_test_split(remaining_paths, remaining_labels, 
                                                                        train_size=val_to_test_ratio, random_state=42,
                                                                       stratify=remaining_labels)

# Preprocesing

In [27]:
train_imgs = []
test_imgs = []
val_imgs = []

clip_limit = 2.0
tile_grid = (8, 8)
img_shape = (299, 299)

clahe = cv2.createCLAHE(clipLimit=clip_limit, tileGridSize=tile_grid)


for path in train_img_paths:
    img = cv2.imread(path)
    img = cv2.resize(img, img_shape[:2])
    lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
    lab[:,:,0] = clahe.apply(lab[:,:,0])
    img = cv2.cvtColor(lab, cv2.COLOR_Lab2RGB)
    train_imgs.append(img)
# print(len(train_imgs))

for path in val_img_paths:
    img = cv2.imread(path)
    img = cv2.resize(img, img_shape)
    lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
    lab[:,:,0] = clahe.apply(lab[:,:,0])
    img = cv2.cvtColor(lab, cv2.COLOR_Lab2RGB)
    val_imgs.append(img)
# print(len(val_imgs))    

for path in test_img_paths:
    img = cv2.imread(path)
    img = cv2.resize(img, img_shape)
    lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
    lab[:,:,0] = clahe.apply(lab[:,:,0])
    img = cv2.cvtColor(lab, cv2.COLOR_Lab2RGB)
    test_imgs.append(img)
# print(len(test_imgs))

# print(train_imgs[0].shape)
# plt.imshow(train_imgs[0], cmap='gray')

### Converting to array

In [28]:
train_imgs = np.asarray(train_imgs)
test_imgs = np.asarray(test_imgs)
val_imgs = np.asarray(val_imgs)

### Normalising the Images

In [29]:
train_imgs = (train_imgs/255.).astype(np.float16)
test_imgs = (test_imgs/255.).astype(np.float16)
val_imgs = (val_imgs/255.).astype(np.float16)

# Model Creation

## Xception model

### Importing Model

In [30]:
import tensorflow as tf
import keras
from keras.applications.xception import Xception
import pickle

2024-05-14 16:41:48.786238: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-14 16:41:48.789612: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-14 16:41:48.876042: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-14 16:41:49.377478: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-14 16:41:51.083978: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF

All the pretrained models need RGB image as an input so image shape must be a tuple of 3 values

In [31]:
img_shape = (img_shape[0], img_shape[1], 3)
num_classes = len(np.unique(labels))
# print(num_classes)

In [32]:
train_labels = keras.utils.to_categorical(train_labels)
test_labels = keras.utils.to_categorical(test_labels)
val_labels = keras.utils.to_categorical(val_labels)

In [33]:
print(np.shape(train_labels))

(4132, 124)


### Initialising the model

In [34]:
base = Xception(include_top=False, weights='imagenet', input_shape=img_shape)

2024-05-14 16:41:53.038392: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-05-14 16:41:53.038962: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


83683744/83683744 ━━━━━━━━━━━━━━━━━━━━ 21s 0us/step


In [35]:
base.summary()

Model: "xception"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 299, 299,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_conv1        │ (None, 149, 149,  │        864 │ input_layer[0][0] │
│ (Conv2D)            │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_conv1_bn     │ (None, 149, 149,  │        128 │ block1_conv1[0][… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_conv1_act    │ (None, 149, 149,  │          0 │ block1_conv1_bn[… │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_conv2        │ (None, 147, 147,  │     18,432 │ block1_conv1_act… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_conv2_bn     │ (None, 147, 147,  │        256 │ block1_conv2[0][… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_conv2_act    │ (None, 147, 147,  │          0 │ block1_conv2_bn[… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2_sepconv1     │ (None, 147, 147,  │      8,768 │ block1_conv2_act… │
│ (SeparableConv2D)   │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2_sepconv1_bn  │ (None, 147, 147,  │        512 │ block2_sepconv1[… │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2_sepconv2_act │ (None, 147, 147,  │          0 │ block2_sepconv1_… │
│ (Activation)        │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2_sepconv2     │ (None, 147, 147,  │     17,536 │ block2_sepconv2_… │
│ (SeparableConv2D)   │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2_sepconv2_bn  │ (None, 147, 147,  │        512 │ block2_sepconv2[… │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 74, 74,    │      8,192 │ block1_conv2_act… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2_pool         │ (None, 74, 74,    │          0 │ block2_sepconv2_… │
│ (MaxPooling2D)      │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 74, 74,    │        512 │ conv2d[0][0]      │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 74, 74,    │          0 │ block2_pool[0][0… │
│                     │ 128)              │            │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block3_sepconv1_act │ (None, 74, 74,    │          0 │ add[0][0]       

 Total params: 20,861,480 (79.58 MB)

 Trainable params: 20,806,952 (79.37 MB)

 Non-trainable params: 54,528 (213.00 KB)

Freezing the weights of the first five layers

In [36]:
for layer in base.layers[:5]:
    layer.trainable = False

In [37]:
x = base.output
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(250, activation='relu')(x)
x = keras.layers.Dropout(0.25)(x)
preds = keras.layers.Dense(num_classes+1, activation='softmax', name='Final')(x)

In [38]:
model = keras.models.Model(inputs=base.input, outputs=preds)

for layer in model.layers:
    layer.trainable = True

# Training

### Making Checkpoints to save the weights periodically

In [39]:
if not os.path.exists(weights_path):
    os.makedirs(weights_path)

In [40]:
checkpoint = keras.callbacks.ModelCheckpoint(filepath=os.path.join(weights_path, 'checkpoint.weights.h5'),
                                           monitor='val_accuracy',
                                           verbose=0,
                                           save_weights_only=True,
                                           save_best_only=True)
# early_stop = keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=6, verbose=1)
reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy', factor=0.1, min_lr=1e-7, 
                                              patience=2, verbose=1)

callbacks = [checkpoint, reduce_lr]

## Setting Parameters

In [41]:
learning_rate = 0.001
epochs = 50
batch_size = 16

## Compile and Train

In [42]:
optimizer = keras.optimizers.Adam(learning_rate=learning_rate)

In [43]:
model.compile(optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(x=train_imgs, y=train_labels, epochs=epochs, batch_size=batch_size, 
                   validation_data = (val_imgs, val_labels), callbacks=callbacks)

Epoch 1/50
116/259 ━━━━━━━━━━━━━━━━━━━━ 23:02 10s/step - accuracy: 0.0041 - loss: 6.0385

KeyboardInterrupt: 

# Evaluation

In [44]:
from sklearn.metrics import mean_squared_error, confusion_matrix
from sklearn.metrics import classification_report

In [45]:
model.load_weights(os.path.join(weights_path, 'best.weights.h5'))

In [46]:
preds = model.predict(test_imgs)
preds = np.argmax(preds, axis=1)
loss = mean_squared_error(np.argmax(test_labels, axis=1), preds)

28/28 ━━━━━━━━━━━━━━━━━━━━ 157s 6s/step


In [47]:
report = classification_report(np.argmax(test_labels, axis=1), preds)
print(report)

              precision    recall  f1-score   support

           1       1.00      1.00      1.00         7
           2       1.00      1.00      1.00         8
           3       0.89      1.00      0.94         8
           4       1.00      1.00      1.00         7
           5       1.00      0.86      0.92         7
           6       1.00      1.00      1.00         7
           7       1.00      1.00      1.00         8
           8       1.00      1.00      1.00         7
           9       1.00      1.00      1.00         7
          10       1.00      1.00      1.00         7
          11       1.00      1.00      1.00         7
          12       1.00      0.86      0.92         7
          13       1.00      0.86      0.92         7
          14       1.00      1.00      1.00         7
          15       0.86      0.86      0.86         7
          16       1.00      1.00      1.00         7
          17       1.00      1.00      1.00         7
          18       0.88    